In [2]:
from litellm import completion
import os

## set ENV variables
# os.environ["OPENAI_API_KEY"] = "your-openai-key"

# os.environ["GEMINI_API_KEY"] = ""
os.environ["TOGETHERAI_API_KEY"] = "dc7b6e35a7a0e0a582905d0c909ed0fb945208a40e25ca8cfee12a1855637b9c"


messages = [{"content": "You are a helpful assistant", "role": "system"}, { "content": "Hello, how are you?","role": "user"}]

# NOTE: adding "together_ai/" in front of the model name in https://docs.together.ai/docs/inference-models before calling.
response = completion(
    model="together_ai/meta-llama/Llama-3-8b-chat-hf", 
    messages=messages,
)



model = "together_ai/meta-llama/Llama-3-8b-chat-hf"
system_prompt = ""
user_prompt = "Q: 1 + 1 = "

# print(togetherai(model, system_prompt, user_prompt))

## framework
- generalize all the methods name

In [3]:
from reasoners.benchmark import gsm8k, aqua, blocksworld, bw_utils, prontoqa
import json


def togetherai(model, system_prompt, user_prompt):
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]
    response = completion(
    model=model, 
    messages=messages,
    )
    return response.choices[0].message.content

class SideReasoner:
    def __init__(self,
                 task,
                 method):
        self.task = task
        self.method = method
        self.evaluator_mapper = {"gsm8k": gsm8k.GSM8KEvaluator,
                                  "aqua": aqua.AQuAEvaluator, 
                                  "blocksworld": blocksworld.BWEvaluator, 
                                  "prontoqa": prontoqa}

    def get_prompt(self, idx = 0, num_shot = 4):
        with open(f'examples/CoT/{self.task}/prompts/{self.method}.json') as f:
            prompt = json.load(f)
        task = self.evaluator_mapper[self.task]
        evaluator = task(output_extractor=None, answer_extractor=None, sample_prompt_type=self.method, init_prompt=prompt)
        prompt = evaluator.sample_prompt(num_shot=num_shot)[self.method]
        question = evaluator.full_dataset[idx]['question']
        answer = evaluator.full_dataset[idx]['answer']

        return prompt.format(QUESTION = question), answer

    def inference(self, model, user_prompt, system_prompt = ""):
        return togetherai(model, system_prompt, user_prompt)



model = "together_ai/meta-llama/Llama-3-8b-chat-hf"
system_prompt = "you are a helpful assistant."
reasoner = SideReasoner(task="gsm8k", method="cot")
prompt, answer = reasoner.get_prompt()
response = reasoner.inference(model=model, user_prompt=prompt, system_prompt=system_prompt)

print(response)

/Users/xiaofeng/Documents/GitHub/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's help Janet with her duck egg sales!

Janet's ducks lay 16 eggs per day. She eats 3 eggs for breakfast, and bakes muffins with 4 eggs, so she uses a total of 3 + 4 = 7 eggs. That leaves 16 - 7 = 9 eggs remaining.

Janet sells these 9 eggs at the farmers' market for $2 per egg. To find out how much she makes, we multiply the number of eggs by the price per egg:

9 eggs x $2 per egg = $18

So, Janet makes $18 every day at the farmers' market.


In [14]:
import datasets
import json
from tqdm import tqdm
import torch
import os, pickle
from datetime import datetime
import sys
import random
import copy

class GSM8K:
    def __init__(self, split = 'test'):
        self.input_processor = lambda x: x["question"]
        self.full_dataset = datasets.load_dataset('gsm8k', 'main', split=split)
        self._dataset_name = 'gsm8k'

    def __str__(self):
        return self._dataset_name

    def get_dataset(self):
        return self.full_dataset
    
tmp = GSM8K()
dataset = tmp.get_dataset()
dataset[0]


{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [ ]:
class Vanilla:
    def __init__(self, dataset, shot):
        self.dataset = dataset
        self.shot = shot

    def get_prompt(self):
        

In [18]:
class Task:
    def __init__(self, model, algorithm, dataset):
        self.model = model
        self.algorithm = algorithm
        self.dataset = dataset

    def inference(shots = 3):
        pass


class Model:
    def __init__(self) -> None:
        pass

    def get_model_api(model):
        return model

    def get_model_local():
        pass



class Algorithm:
    def __init__(self):
        pass
        
    def vanilla():
        return ""

    def chain_of_thought():
        return ""

    def chain_of_thought_zero_shot():
        return "let's think step by step..."


class Dataset:
    def __init__(self,
                 dataset,
                 metrics=None):
        self.dataset = dataset
        self.metrics = metrics
        self.dataset_mapper = {"gsm8k": GSM8K}

    def get_dataset(self):
        data_class = self.dataset_mapper[self.dataset]()
        return data_class.get_dataset()


    # def get_example(self, cnt = 3):      
    #     pass


dataset = Dataset(dataset="gsm8k")
tmp = dataset.get_dataset()
tmp[0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}